In [1]:
import numpy as np
import pandas as pd
import math
from datetime import time
import CMAPSAuxFunctions

from data_handler_VALVE import ValveDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

Using TensorFlow backend.


<h1> Create Data Handler </h1>

In [2]:
features = ['timestamp', 'externalControllerOutput', 'undisturbedMediumFlow', 'pressureValveInlet', 'pressureValveOutlet', 'mediumTemperature', 'rodDisplacement', 'disturbedMediumFlow', 'selectedFault', 'faultType', 'faultIntensity']
selected_indices = np.array([2,4,5,6,7,8])
selected_features = list(features[i] for i in selected_indices-1)

window_size = 30
window_stride = 1

# min = 2018-02-14 18:59:20
# max = 2018-08-19 18:28:20
time_start = "2018-02-14 18:59:20"
time_end = "2018-04-19 18:28:20"

# Either classification or regression
problem = 'classification'
#problem = 'regression'

vHandler = ValveDataHandler(time_start, time_end, selected_features = selected_features,
                            sequence_length = window_size, sequence_stride = window_stride,
                            problem = problem)

init


<h1> Keras Model </h1>

In [3]:
#np.set_printoptions(threshold=np.nan)

K.clear_session()
lambda_regularization = 0.20

def create_ANN_model(input_shape, problem):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim = input_shape, activation = 'relu', kernel_initializer = 'glorot_normal', 
                    kernel_regularizer = regularizers.l2(lambda_regularization), name = 'fc1'))
    
    if (problem == 'classification'):
        model.add(Dense(1, activation = 'softmax', name = 'out'))
    elif (problem == 'regression'):
        model.add(Dense(1, activation = 'linear', name = 'out'))
        
    #model.add(Dense(1, activation = 'softmax', name = 'out'))
    
    return model

<h1>Tunable Model </h1>

In [4]:
scaler = MinMaxScaler(feature_range = (-1, 1))
#scaler = StandardScaler()

In [5]:
def get_compiled_model(shape, problem):
    
    K.clear_session()
    
    if (problem == 'classification'):
        # Parameters for the model
        optimizer = SGD(lr = 0.01, momentum = 0.0, decay = 0.0, nesterov = False)
        loss_function = 'categorical_crossentropy'
        metrics = ['accuracy']
    elif (problem == 'regression'):
        # Parameters for the model
        optimizer = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = False)
        loss_function = 'mean_squared_error'
        metrics = ['mse']
        
    model = None
    
    # Create and compile the model
    model = create_ANN_model(shape, problem)
    model.compile(optimizer = optimizer, loss = loss_function, metrics = metrics)
    
    return model

In [6]:
num_features = len(selected_features)
input_shape = num_features * window_size

model = get_compiled_model(input_shape, problem)
tModel = SequenceTunableModelRegression('ANN_Model', model, lib_type = 'keras', data_handler = vHandler)

tModel.data_handler.data_scaler = None
tModel.data_scaler = scaler

<h1> Loading Data from MySQL Database </h1>

In [7]:
#vHandler.connect_to_db("remoteAdmin","remoteAdmin","169.236.181.40:3306","damadics")

In [8]:
# Extract data from database
#vHandler.extract_data_from_db()
# vHandler.extract_data_from_db()

In [9]:
# Extract data from .csv file. If loading from .csv, don't forget to uncomment .extract_data_from_db() in data_handler_VALVE'''
vHandler._df = pd.read_csv('valve_dataset.csv', sep = ',')
for i in range(1,9):
    vHandler._df[vHandler._df.columns[-i]] = vHandler._df[vHandler._df.columns[-i-1]]
vHandler._df['undisturbedMediumFlow'] = pd.np.nan

# adds a new feature called "Status" on whether the valve is broken '1' or not broken '0'
#vHandler._df['status'] = vHandler._df['selectedFault'].apply(lambda valve: 0 if valve == 20 else 1)
# this adds a column to vHandler._X
#vHandler._X = vHandler._df[selected_features]
#vHandler._X.insert(loc = 0, column = 'id', value = np.arange(len(vHandler._df)))
# vHandler._X = vHandler._X.values

# vHandlezr._y = vHandler._df['status'].values
# print(vHandler._df.head())

In [10]:
# vHandler._df['id'] = np.arange(1, vHandler._df.shape[0] + 1)
# print(vHandler._df)

In [11]:
vHandler._X = vHandler._df[selected_features].values

if (problem == 'classification'):
    vHandler._df['status'] = vHandler._df['selectedFault'].apply(lambda valve: 1 if valve == 20 else -1)
    vHandler._y = vHandler._df["status"].values
elif (problem == 'regression'):
    vHandler._y = vHandler._df["selectedFault"].values
        
# vHandler._X = vHandler._df[selected_features].values
# vHandler._y = vHandler._df["selectedFault"].values
#vHandler._y = list(map(lambda valve: 0 if valve == 20 else 1, vHandler._y))
#vHandler._y = vHandler._df["status"].values

In [12]:
# print(vHandler._X)
# print(vHandler._y)

In [13]:
#vHandler.load_data(cross_validation_ratio = 0.3, test_ratio = 0.2, unroll = True)
tModel.load_data(unroll = True, verbose = 0, cross_validation_ratio = 0.2, test_ratio = 0.2)

Loading data from database
265100
Splitting into samples: 0:00:00.956568
counter: 129
265100
Splitting into samples: 0:00:01.253910
counter: 129
Train, cv, and test splitting: 0:00:00.000337
Sequence length 30
Sequence stride 1
X_train len 79
X_crossVal len 25
X_test len 25
y_train len 79
y_crossVal len 25
y_test len 25
X_train[0]
(1641, 6)
[[7.32444e-01 8.51909e-01 6.57137e-01 2.14812e-01 9.99634e-01 4.84000e-04]
 [2.57854e-01 8.48337e-01 6.53788e-01 2.14869e-01 1.00000e+00 0.00000e+00]
 [6.59356e-01 8.48023e-01 6.53302e-01 2.13990e-01 9.99705e-01 1.16900e-03]
 ...
 [3.66043e-01 8.49039e-01 6.55951e-01 2.14745e-01 9.99418e-01 0.00000e+00]
 [7.32444e-01 8.51408e-01 6.52105e-01 2.15247e-01 9.97463e-01 0.00000e+00]
 [2.57854e-01 8.47758e-01 6.51201e-01 2.16029e-01 9.97559e-01 4.91000e-04]]
X_crossVal[0]
(30, 6)
[[0.257854 0.849312 0.647208 0.215167 0.479946 0.770706]
 [0.659356 0.849352 0.651728 0.215911 0.817959 0.142601]
 [0.484302 0.850423 0.655458 0.214001 0.469989 0.797355]
 [0.3660

In [14]:
print(tModel.data_handler._X_crossVal_list)

[array([[0.257854, 0.849312, 0.647208, 0.215167, 0.479946, 0.770706],
       [0.659356, 0.849352, 0.651728, 0.215911, 0.817959, 0.142601],
       [0.484302, 0.850423, 0.655458, 0.214001, 0.469989, 0.797355],
       [0.366043, 0.847968, 0.655359, 0.212166, 0.673319, 0.288347],
       [0.732444, 0.850535, 0.654552, 0.215271, 0.735481, 0.208158],
       [0.257854, 0.849892, 0.652395, 0.213396, 0.505362, 0.676154],
       [0.659356, 0.848152, 0.6469  , 0.215588, 0.824219, 0.137178],
       [0.484302, 0.849864, 0.648563, 0.216226, 0.461118, 0.860357],
       [0.366043, 0.848572, 0.645768, 0.2165  , 0.648275, 0.335436],
       [0.732444, 0.851345, 0.642471, 0.212673, 0.726778, 0.227471],
       [0.257854, 0.851455, 0.645969, 0.217109, 0.47718 , 0.782473],
       [0.659356, 0.848963, 0.653183, 0.214609, 0.812669, 0.14463 ],
       [0.484302, 0.848876, 0.652345, 0.215286, 0.46742 , 0.803757],
       [0.366043, 0.850336, 0.657494, 0.21782 , 0.67398 , 0.285509],
       [0.732444, 0.849478, 0.658

In [15]:
## if (problem == 'classification'):
    broken_counter = 0
    not_broken_counter = 0
    dataset = vHandler._y_crossVal
    print(dataset.shape)
    #print(vHandler._X_test[1])
    # print(vHandler._y_test)

    for i in dataset:
        #print(i)
        if (i == 1.0):
            not_broken_counter += 1;
        else:
            broken_counter += 1;

    print("Broken Counter:", broken_counter)
    print("Not Broken Counter:", not_broken_counter)

    broken_counter = 0
    not_broken_counter = 0
    dataset = vHandler._y_test
    print(dataset.shape)
    #print(vHandler._X_test[1])
    # print(vHandler._y_test)

    for i in dataset:
        #print(i)
        if (i == 1.0):
            not_broken_counter += 1;
        else:
            broken_counter += 1;

    print("Broken Counter:", broken_counter)
    print("Not Broken Counter:", not_broken_counter)

(25, 1)
Broken Counter: 7
Not Broken Counter: 18
(25, 1)
Broken Counter: 6
Not Broken Counter: 19


In [16]:
vHandler.print_data(True)

# tModel = SequenceTunableModelRegression('ANN_Model', model, lib_type = 'keras', data_handler = vHandler)

Printing shapes

Training data (X, y)
(155239, 180)
(155239, 1)
Cross-Validation data (X, y)
(25, 180)
(25, 1)
Testing data (X, y)
(25, 180)
(25, 1)
Printing first 5 elements

Training data (X, y)
[[7.32444e-01 8.51909e-01 6.57137e-01 2.14812e-01 9.99634e-01 4.84000e-04
  2.57854e-01 8.48337e-01 6.53788e-01 2.14869e-01 1.00000e+00 0.00000e+00
  6.59356e-01 8.48023e-01 6.53302e-01 2.13990e-01 9.99705e-01 1.16900e-03
  4.84302e-01 8.49598e-01 6.48753e-01 2.15082e-01 1.00000e+00 9.67000e-04
  3.66043e-01 8.46974e-01 6.44832e-01 2.18243e-01 9.99396e-01 0.00000e+00
  7.32444e-01 8.48532e-01 6.42809e-01 2.13921e-01 1.00000e+00 5.40000e-05
  2.57854e-01 8.51747e-01 6.44124e-01 2.14866e-01 1.00000e+00 1.08200e-03
  6.59356e-01 8.47455e-01 6.45996e-01 2.12393e-01 1.00000e+00 0.00000e+00
  4.84302e-01 8.49973e-01 6.51102e-01 2.17505e-01 1.00000e+00 6.50000e-04
  3.66043e-01 8.50311e-01 6.55447e-01 2.15010e-01 1.00000e+00 0.00000e+00
  7.32444e-01 8.48040e-01 6.55764e-01 2.14973e-01 9.97137e-01 0

<h1> Local Outlier Factor </h1>

In [17]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
LOC = NearestNeighbors(n_neighbors = 20, leaf_size = 30)
LOC.fit(tModel.data_handler._X_train)

In [ ]:
y_pred_crossVal = LOC.kneighbors(tModel.data_handler._X_crossVal)
# y_pred_test = LOC.predict(tModel.data_handler._X_test)

In [ ]:
print(y_pred_crossVal[0])

<h1> Isolation Forest </h1>

In [18]:
from sklearn.ensemble import IsolationForest

In [35]:
iso_forest = IsolationForest(n_estimators = 200, max_samples = 'auto', contamination = 'auto',
                             max_features = 1.0, bootstrap = False, n_jobs = None, behaviour = 'new',
                             random_state = None, verbose = 1)
iso_forest.fit(tModel.data_handler._X_train)

/home/cmartin28/anaconda3/envs/work/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:214: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.6min finished


IsolationForest(behaviour='new', bootstrap=False, contamination='legacy',
        max_features=1.0, max_samples='auto', n_estimators=200,
        n_jobs=None, random_state=None, verbose=1)

In [36]:
# iso_forest.decision_function(tModel.data_handler._X_train)

In [37]:
y_pred_crossVal = iso_forest.predict(tModel.data_handler._X_crossVal)
y_pred_test = iso_forest.predict(tModel.data_handler._X_test)

In [38]:
acc_crossVal = metrics.accuracy_score(tModel.data_handler._y_crossVal, y_pred_crossVal)
precision_crossVal = metrics.precision_score(tModel.data_handler._y_crossVal, y_pred_crossVal)
recall_crossVal = metrics.recall_score(tModel.data_handler._y_crossVal, y_pred_crossVal)
confusion_matrix_crossVal = metrics.confusion_matrix(tModel.data_handler._y_crossVal, y_pred_crossVal)

print("Accuracy for cross-validation:", acc_crossVal)
print("Precision for cross-validation:", precision_crossVal)
print("Recall for cross-validation:", recall_crossVal)
print("Confusion matrix for cross-validation\n", confusion_matrix_crossVal)

Accuracy for cross-validation: 0.8
Precision for cross-validation: 0.782608695652174
Recall for cross-validation: 1.0
Confusion matrix for cross-validation
 [[ 2  5]
 [ 0 18]]


In [39]:
acc_test = metrics.accuracy_score(tModel.data_handler._y_test, y_pred_test)
precision_test = metrics.precision_score(tModel.data_handler._y_test, y_pred_test)
recall_test = metrics.recall_score(tModel.data_handler._y_test, y_pred_test)
confusion_matrix_test = metrics.confusion_matrix(tModel.data_handler._y_test, y_pred_test)

print("Accuracy for test:", acc_test)
print("Precision for test:", precision_test)
print("Recall for test:", recall_test)
print("Confusion matrix for test\n", confusion_matrix_test)

Accuracy for test: 0.72
Precision for test: 0.75
Recall for test: 0.9473684210526315
Confusion matrix for test
 [[ 0  6]
 [ 1 18]]


In [34]:
del iso_forest